In [40]:
from base.SushiGo.env import *
import time
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
import random

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
list_player = [player_random]*4
normal_main(list_player, 1, [0])

([0, 1, 0, 0], [0])

In [43]:
a = time.time()
normal_main(list_player, 10000, [0])
b = time.time()
print(b-a)
c = time.time()
for i in range(10000):
    normal_main(list_player, 1, [0])
d = time.time()
print(d-c)

13.65406847000122
13.722010850906372


In [1]:
import numpy as np
import random
import os

In [39]:
list_all_player = os.listdir('Agent')
print(list_all_player)
list(np.random.choice(list_all_player, size = 5, replace = False))

['Agent_best', 'An_200922', 'Dat_130922', 'Dat_200922', 'Hieu_130922', 'Hieu_200922', 'KPI_200922', 'NhatAnh_200922', 'Phong_130922', 'Phong_200922', 'player_random', 'Trang']


['Agent_best', 'Phong_130922', 'Phong_200922', 'Dat_130922', 'Hieu_200922']

In [66]:
import numpy as np
np.random.uniform(0.01, 0.09)

0.024181676462718474

In [62]:
import numpy as np
import itertools
import random
import copy
import time
import warnings 
from numba import njit
warnings.filterwarnings('ignore')

import os
import sys
from setup import game_name,time_run_game
sys.path.append(os.path.abspath(f"base/{game_name}"))
from env import *

player = 'Phong_130922'
path_data = f'Agent/{player}/Data'
if not os.path.exists(path_data):
    os.mkdir(path_data)
path_save_player = f'Agent/{player}/Data/{game_name}_{time_run_game}/'
if not os.path.exists(path_save_player):
    os.mkdir(path_save_player)


class Phong_att():
    def __init__(self):
        self.layer = 1
        self.count_matrix = 5
        self.count_test_matrix = 1
        self.add_reward = 1.2
        self.sub_reward = 0.8
        self.percent_matrix_choice = 0.8
        self.count_matrix_remove = 0.6


def player_random(state, temp, per):
    actions = get_list_action(state)
    action = random.choice(actions)
    return action, temp, per

# @njit()
def file_temp_to_action(state, file_temp, layer):
    a = get_list_action(state)
    RELU = np.ones(len(state))
    for i in range(layer-1):
        state1 = np.dot(state,file_temp[0][i])
        state1 -= file_temp[1][i]
        RELU = state1*(state1>0)
    matrix_new = np.dot(RELU,file_temp[0][-1])
    list_val_action = matrix_new[a]
    action = a[np.argmax(list_val_action)]
    return action
    


def activation_func(state,file_temp,file_per):
    if file_per == [0]:
        file_per = []
    if len(file_temp)  < 2:
        list_matrix = []
        list_bius = []
        for i in range(Phong_att().layer):
            list_matrix.append(np.random.random((len(state), len(state)))*2-Phong_att().count_matrix_remove)
            list_bius.append([random.uniform(-1, 1) for x in range(len(state))])
        list_matrix.append(np.random.random((len(state), amount_action()))*2-Phong_att().count_matrix_remove)
        file_temp = [list_matrix, list_bius]
    action = file_temp_to_action(state, file_temp, Phong_att().layer)
    if check_victory(state) == 1:
        if len(file_per) == 0:
            file_per = [file_temp]
        else:
            file_per.append(file_temp)
    return action,file_temp,file_per

def choose_matrix(state,file_temp,file_per, id_matrix):
    if len(file_temp) < 2:
        file_temp = [file_per[0][id_matrix], id_matrix]
    action = file_temp_to_action(state, file_temp[0], Phong_att().layer)

    check_vic = check_victory(state)
    if check_vic == 1:     
        file_per[1][file_temp[1]] += Phong_att().add_reward
    if check_vic == 0:     
        file_per[1][file_temp[1]] -= Phong_att().sub_reward
        
    return action,file_temp,file_per

def matrix1(state,file_temp,file_per):
    action,file_temp,file_per = choose_matrix(state,file_temp,file_per, file_per[2][0])
    return action,file_temp,file_per
def matrix2(state,file_temp,file_per):
    action,file_temp,file_per = choose_matrix(state,file_temp,file_per, file_per[2][1])
    return action,file_temp,file_per
def matrix3(state,file_temp,file_per):
    action,file_temp,file_per = choose_matrix(state,file_temp,file_per, file_per[2][2])
    return action,file_temp,file_per
def matrix4(state,file_temp,file_per):
    action,file_temp,file_per = choose_matrix(state,file_temp,file_per, file_per[2][3])
    return action,file_temp,file_per
def matrix5(state,file_temp,file_per):
    action,file_temp,file_per = choose_matrix(state,file_temp,file_per, file_per[2][4])
    return action,file_temp,file_per



def phong_create_list_matrix(file_per_save_data, type_train):
    while len(file_per_save_data) < amount_player():
        count, file_per_save_data = normal_main([activation_func]*amount_player(),Phong_att().count_matrix,[0])

    list_combinations = [list(i) for i in list(itertools.combinations(list(range(len(file_per_save_data))), amount_player()))]
    file_per_save_data = [file_per_save_data, [-99 for i in range(len(file_per_save_data))], []]
    for list_after_combinations in list_combinations:
        file_per_save_data[2] = list_after_combinations
        if amount_player() == 4:
            list_player = [matrix1, matrix2, matrix3, matrix4]
        elif amount_player() == 5:
            list_player = [matrix1, matrix2, matrix3, matrix4, matrix5]
        else:
            list_player = [matrix1, matrix2, matrix3, matrix4] + player_random*(amount_player()-1)
        count, file_per_save_data = normal_main(list_player, Phong_att().count_test_matrix,file_per_save_data)

    if type_train == 1:

        file_per_save_data[1] = np.array(file_per_save_data[1])
        file_per_save_data[0] = np.array(file_per_save_data[0])

        arg_sort = np.argsort(file_per_save_data[1])
        lst_argmax = arg_sort[int(len(file_per_save_data[1])*(Phong_att().percent_matrix_choice)):]
        return list(file_per_save_data[0][lst_argmax]), file_per_save_data[0][np.argmax(file_per_save_data[1])]

    if type_train == 2:
        return file_per_save_data[0][np.argmax(file_per_save_data[1])]

# @njit()
def save_file(check_point_save_file_all, p_check_point, p_bot_save):
    if check_point_save_file_all < p_check_point:
        check_point_save_file_all = p_check_point
        # print('checkpoint',check_point_save_file_all)
        np.save(f'{path_save_player}p_model_manh_nhat.npy', [p_bot_save])
    return check_point_save_file_all


def find_best_model(check_point_save_file_all):
    list_maxtrix_max_all = []

    p_check_point_1 = 1
    p_check_point_2 = 2
    p_check_point_3 = 3
    p_check_point_4 = 4
    p_check_point_5 = 5
    while len(list_maxtrix_max_all) < Phong_att().count_matrix:
        list_matrix_max_3 = []
        while len(list_matrix_max_3) < Phong_att().count_matrix:
            list_matrix_max_2 = []
            while len(list_matrix_max_2) < Phong_att().count_matrix:
                print('            Tứ kết: ', len(list_matrix_max_2), end = '')
                list_matrix_max_1 = []
                print(' Vòng bảng:', end = ' ')
                while len(list_matrix_max_1) < Phong_att().count_matrix:
                    print(len(list_matrix_max_1), end = ' ')
                    file_per_save = []
                    list_file_per_save, bot_manh_nhat_temp = phong_create_list_matrix(file_per_save, 1)
                    check_point_save_file_all = save_file(check_point_save_file_all, p_check_point_1, bot_manh_nhat_temp) #checkpoint1, vòng bảng
                    list_matrix_max_1 += list_file_per_save

                list_file_per_save, bot_manh_nhat_temp = phong_create_list_matrix(list_matrix_max_1, 1)
                check_point_save_file_all = save_file(check_point_save_file_all, p_check_point_2, bot_manh_nhat_temp) #checkpoint2, tứ kết
                list_matrix_max_2 += list_file_per_save
                # print('')
            list_file_per_save, bot_manh_nhat_temp = phong_create_list_matrix(list_matrix_max_2, 1)
            list_matrix_max_3 += list_file_per_save
            check_point_save_file_all = save_file(check_point_save_file_all, p_check_point_3, bot_manh_nhat_temp) #checkpoint3, bán kết
            print('      Bán kết: ', len(list_matrix_max_3))

        list_file_per_save, bot_manh_nhat_temp = phong_create_list_matrix(list_matrix_max_3, 1)
        check_point_save_file_all = save_file(check_point_save_file_all, p_check_point_4, bot_manh_nhat_temp) #checkpoint4, chung kết
        list_maxtrix_max_all += list_file_per_save
        print('   Chung kết: ', len(list_maxtrix_max_all))
    print('**************')
    bot_manh_nhat  = phong_create_list_matrix(list_maxtrix_max_all, 2)
    print('Tìm xong nhà vô địch')
    check_point_save_file_all = save_file(check_point_save_file_all, p_check_point_5, bot_manh_nhat) #checkpoint5
    return bot_manh_nhat, check_point_save_file_all


def p_test_matrix_old(state,file_temp,file_per):
    file_temp = file_per[0]
    action = file_temp_to_action(state, file_temp, Phong_att().layer)
    return action,file_temp,file_per

def p_test_matrix_new(state,file_temp,file_per):
    file_temp = file_per[1]
    action = file_temp_to_action(state, file_temp, Phong_att().layer)
    return action,file_temp,file_per


# tối ưu ở đây, amount model = a
# sau khi tìm được ma trận mạnh hơn thì gán luôn
# thay vì chơi với random thì chơi với người a[0] + a[1] - a[2] - a[3]
# ở phần test thay vì thua thì đổi random thì đổi có chọn lọc
# hoặc có thể tính theo số trận thắng, nhiều hơn thì chọn model đó
# thêm numpy vào mốt số hàm có thể sử dụng được
def p_train_lan_2(n, a):
    if len(a) > 0:
        amount_model = len(a)
        p_len_state = len(a[0][0][0])
        model_manh_nhat = a
    else:
        model_manh_nhat = np.load(f'{path_save_player}p_model_manh_nhat.npy', allow_pickle=True)
        amount_model = len(model_manh_nhat)
        p_len_state = len(model_manh_nhat[0][0][0])
    for change in range(n):
        for phong_id_model_choose in range(amount_model):
            matrix_remove = (np.random.random((p_len_state, amount_action()))*2 - 1)*0.05
            model_manh_nhat_new = copy.deepcopy(model_manh_nhat)
            model_manh_nhat_new[phong_id_model_choose][0][1] = model_manh_nhat[phong_id_model_choose][0][1] - matrix_remove

            list_player = [p_test_matrix_old, p_test_matrix_old, p_test_matrix_new, p_test_matrix_new] + [player_random]*int(amount_player()-4)
            count, _ = normal_main(list_player,1000,[model_manh_nhat[phong_id_model_choose], model_manh_nhat_new[phong_id_model_choose]])
            if (count[1] + count[0]) - ( count[3] + count[2]) >= 20:
                print('thang', count, 'phong_id_model_choose',  phong_id_model_choose, 'change', change, 'test voi bot', count_)
                list_player = [p_test_matrix_new] + [player_random]*int(amount_player()-1)
                count_, _= normal_main(list_player,1000,[model_manh_nhat[phong_id_model_choose], model_manh_nhat_new[phong_id_model_choose]])
                np.save(f'{path_save_player}p_model_manh_nhat.npy', model_manh_nhat_new)
                model_manh_nhat = copy.deepcopy(model_manh_nhat_new)

def train(n):
    np.save(f'{path_save_player}phong_id_model_choose.npy', 0)
    a = []
    check_point_save_file_all = 0
    for i in range(3):
        start = time.time()
        phong_find_best_model, check_point_save_file_all = find_best_model(check_point_save_file_all)
        end = time.time()
        a.append(phong_find_best_model)
        np.save(f'{path_save_player}p_model_manh_nhat.npy', a)
        print(len(a), end - start)
    p_train_lan_2(n, a)
    print('***Phong Train xong***')


def test(state,file_temp,file_per):
    if len(file_temp) < 2:
        player = 'Phong_130922'
        path_save_player = f'Agent/{player}/Data/{game_name}_{time_run_game}/'
        model_manh_nhat = np.load(f'{path_save_player}p_model_manh_nhat.npy', allow_pickle=True)
        phong_id_model_choose = int(np.load(f'{path_save_player}phong_id_model_choose.npy'))
        file_temp = model_manh_nhat[phong_id_model_choose]
        # file_temp = file_temp.append(phong_id_model_choose)
    action = file_temp_to_action(state, file_temp, Phong_att().layer)

    if check_victory(state) == 0:
        player = 'Phong_130922'
        path_save_player = f'Agent/{player}/Data/{game_name}_{time_run_game}/'
        model_manh_nhat = np.load(f'{path_save_player}p_model_manh_nhat.npy', allow_pickle=True)
        if len(model_manh_nhat) > 1:
            phong_list_model = list(range(len(model_manh_nhat)))
            current_id = int(np.load(f'{path_save_player}phong_id_model_choose.npy'))
            phong_list_model.remove(current_id)
            phong_id_model_choose = random.choice(phong_list_model)
            # print('id', current_id , 'đổi thành ',phong_id_model_choose)
            np.save(f'{path_save_player}phong_id_model_choose.npy', phong_id_model_choose)
    return action,file_temp,file_per



In [63]:
train(1)

            Tứ kết:  0 Vòng bảng: 0 

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function isub>) found for signature:
 
 >>> isub(array(float64, 1d, C), array(float64, 2d, C))
 
There are 10 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'NumpyRulesInplaceArrayOperator.generic': File: numba\core\typing\npydecl.py: Line 244.
    With argument(s): '(array(float64, 1d, C), array(float64, 2d, C))':[0m
[1m   Rejected as the implementation raised a specific error:
     AttributeError: 'NoneType' object has no attribute 'args'[0m
  raised from c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\typing\npydecl.py:255
[1m  - Of which 6 did not match due to:
  Overload of function 'isub': File: <numerous>: Line N/A.
    With argument(s): '(array(float64, 1d, C), array(float64, 2d, C))':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  Operator Overloa

In [128]:
@njit()
def one_game(list_player, temp_file, per_file):
    env, lv1, lv2, lv3 = Reset()
    _cc = 0
    while env[100] <= 400 and _cc <= 10000:
        p_idx = env[100]%4
        p_state = get_player_state(env, lv1, lv2, lv3)
        act, temp_file[p_idx], per_file = list_player[p_idx](p_state, temp_file[p_idx], per_file)
        env, lv1, lv2, lv3 = step(act, env, lv1, lv2, lv3)

        if close_game(env) != 0:
            break

        _cc += 1
    
    turn = env[100]
    for i in range(4):
        env[100] = i
        act, temp_file[i], per_file = list_player[i](get_player_state(env, lv1, lv2, lv3), temp_file[i], per_file)
    
    env[100] = turn
    return close_game(env), per_file

@njit()
def player_random(p_state, temp_file, per_file):
    arr_action = get_list_action(p_state)
    act_idx = np.random.randint(0, len(arr_action))
    return arr_action[act_idx], temp_file, per_file
    
@njit()
def normal_main(list_player, num_game, temp_file, per_file):
    if len(list_player) != 4:
        print('Game chỉ cho phép có đúng 4 người chơi')
        return [-1,-1,-1,-1,-1], per_file
    
    num_won = [0,0,0,0,0]
    p_lst_idx = np.array([0,1,2,3])
    # temp_file = [np.array([0]),np.array([1]),np.array([2]),np.array([3])]
    for _n in range(num_game):
        np.random.shuffle(p_lst_idx)
        temp_file_new = [temp_file[p_lst_idx[i]] for i in range(amount_player())]
        list_player_new = [list_player[p_lst_idx[i]] for i in range(amount_player())]
        # print(temp_file_new, p_lst_idx)
        winner,per_file = one_game(
            list_player_new, temp_file_new, per_file,
        )
        # temp_file = [temp_file_new[np.where(p_lst_idx == i)[0][0]] for i in range(amount_player())]
        # print(temp_file)
        if winner != 0:
            num_won[p_lst_idx[winner-1]] += 1
        else:
            num_won[4] += 1

    return num_won, per_file


# temp_file = [np.array([np.array([2]), np.array([21])]),np.array([np.array([2]), np.array([21])]),np.array([np.array([2]), np.array([21])]),np.array([np.array([2]), np.array([21])])]
file_temp = [(np.random.random((30,100)))*2 -0.67,np.random.random((100,20))]
temp_file = file_temp*4
list_player = [player_random]*4
normal_main(list_player, 1, temp_file, [0])

([0, 0, 1, 0, 0], [0])

In [114]:
@njit()
def sigmoid(x:np.float64) -> np.float64:
    sig = 1 / (1 + np.exp(-x))
    return sig

@njit()
def silu(x:np.float64, theda = 1.0) -> np.float64:
    return x * sigmoid(theda *x)
    
@njit()
def neural_network(play_state:np.float64, file_temp) -> np.float64:
    # play_state = np.array(play_state)
    if 55 < len(play_state) < 70 or len(play_state) > 300 : # TLMN , TLMN_v2 , CENTURY
        matran1 = np.dot(play_state,file_temp[0])
        matran1 = 1 / (1 + np.exp(-matran1))
        matran21 = np.dot(matran1,file_temp[1])
        matran21 *= (matran21 > 0)
        matran2 = np.dot(matran21, file_temp[2])
        return matran2.astype(np.float64)
    elif 120 <len(play_state)  < 170:# SPLENDOr SPlendor_view_only
        matrix1 = np.dot(play_state, file_temp[0])
        matrixRL1 = 1 / (1 + np.exp(-matrix1))
        matrix2 = np.dot(matrixRL1, file_temp[1])
        matrixRL2 = 1 / (1 + np.exp(-matrix2))
        all_action_val = np.dot(matrixRL2, file_temp[2])
        return all_action_val.astype(np.float64)  
    elif 170 < len(play_state) < 250  : #SHERIFF 
        matran1 = np.dot(play_state, file_temp[0])
        matran1 = silu(matran1, theda = 1.0)
        matran2 = np.dot(matran1, file_temp[1])
        return matran2.astype(np.float64)
    else :#SUSHIGO-main, MACHIKOR0
        matran1 = np.dot(play_state, file_temp[0])
        matran1 = silu(matran1, theda = 1.0)
        matran2 = np.dot(matran1, file_temp[1])
        return matran2.astype(np.float64)

print(neural_network(play_state, file_temp))

[26.15221758 28.09878304 23.1691043  27.51775087 26.17713021 26.22732579
 23.29809346 23.45173179 23.99868606 23.58770622 23.59198989 30.29923611
 22.4943759  24.98688892 22.51630581 25.84657074 26.23634392 23.65525294
 21.30323642 26.62585134 25.24234263 25.58862942 24.15340634 28.33877959
 23.30418555 24.24932673 26.80601454 27.2788818  24.40066884 27.71530405
 23.70690769 23.90694869 26.93788679 26.28620041 26.48822578 28.58454288
 25.24356515 21.61571081 24.07985755 23.3489784  23.72704608 26.54160754]


In [132]:
@njit()
def random_player(p_state, temp_file, per_file):
    arr_action = get_list_action(p_state)
    act_idx = np.random.randint(0, len(arr_action))
    return arr_action[act_idx], temp_file, per_file
    
@njit()
def playerRandom1(play_state:np.float64,file_temp:list,file_per:list):
    # print(type(file_temp))
    a = get_list_action(play_state)
    # file_temp = [(np.random.random((len(play_state),100)))*2 -0.67,np.random.random((100,amount_action()))]
    # print(file_temp, file_per)
    # raise 1
    matran2 = neural_network(play_state, file_temp)
    # max = 0
    # action_max = a[random.randrange(len(a))]
    # for act in a:
    #     if matran2[act] > max:
    #         max = matran2[act]
    #         action_max = act

    # check_vic = check_victory(play_state)
    # if check_vic == 1:
    #     if type(file_per[0]) == int:
    #         file_per = [file_temp]
    #     else:
    #         file_per.append(file_temp)
    return int(a[0]),file_temp,file_per
    
# play_state = np.array([7.0, 7.0, 7.0, 7.0, 7.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 4.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 4.0, 2.0, 2.0, 4.0, 5.0, 0.0, 0.0, 3.0, 0.0, 2.0, 1.0, 0.0, 5.0, 0.0, 0.0, 0.0, 2.0, 4.0, 5.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 5.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 0.0, 3.0, 5.0, 5.0, 1.0, 0.0, 3.0, 0.0, 0.0, 7.0, 4.0, 3.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 3.0, 3.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 3.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 12.0])
# temp1 = (np.random.random((len(play_state),100)))*2 -0.6
# temp2 = np.random.random((100, 50))
# temp3 = np.random.random((50,amount_action()))
file_temp = [(np.random.random((len(play_state),100)))*2 -0.67,np.random.random((100,amount_action()))]
list_player= [playerRandom1]*4
temp_file = file_temp*4
kq, file_ = normal_main(list_player, 1, temp_file, np.array([0]))
print(kq)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m[1m[1m[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function dot at 0x0000022C269A1B40>) found for signature:
 
 >>> dot(float64, array(float64, 1d, C))
 
There are 4 candidate implementations:
[1m     - Of which 4 did not match due to:
     Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba\core\overload_glue.py: Line 131.
       With argument(s): '(float64, array(float64, 1d, C))':[0m
[1m      Rejected as the implementation raised a specific error:
        TypingError: Failed in nopython mode pipeline (step: nopython frontend)
      [1m[1m[1mNo implementation of function Function(<intrinsic stub>) found for signature:
       
      

In [16]:
import random
import numpy as np

path_save_player = f'test/'



def playerScore1(play_state,file_temp,file_per):
    a = get_list_action(play_state)

    if len(file_per) < 2:
        file_ = np.load(f'{path_save_player}CK_Matran.npy',allow_pickle=True)
        file_per = [file_, np.zeros(len(file_))]
    
    if len(file_temp) < 2:
        Rand = random.randrange(len(file_per[0]))
        file_temp = [file_per[0][Rand],Rand]
        
    matran2 = neural_network(play_state, file_temp[0])

    max = 0
    action_max = a[random.randrange(len(a))]
    for act in a:
        if matran2[act] > max:
            max = matran2[act]
            action_max = act

    if check_victory(play_state) == 1:
        file_per[1][file_temp[1]] += 1.2
    if check_victory(play_state) == 0:
        file_per[1][file_temp[1]] -= 0.8
    return action_max,file_temp,file_per

# Hàm test và train 
def test(play_state,file_temp,file_per):
    path_save_player = f'test/'
    #Sau đó đọc file test như bình thường
    a = get_list_action(play_state)
    
    if len(file_temp) < 2:
        file_temp = np.load(f'{path_save_player}CK_Win.npy',allow_pickle=True)

    matran2 = neural_network(play_state, file_temp)

    max_ = 0
    action_max = a[random.randrange(len(a))]
    
    for act in a:
        if matran2[act] > max_:
            max_ = matran2[act]
            action_max = act
    return action_max,file_temp,file_per


def train(number_tran):
    list_player= [playerRandom1]*3 + [random_player]*(amount_player()-3)
    kq, file_ = normal_main(list_player, 1500, [0])
    np.save(f'{path_save_player}CK_Matran.npy',file_)

    list_player= [playerScore1]*3 + [random_player]*(amount_player()-3)
    kq, file_2 = normal_main(list_player,3500, [0])
    matran_Win= file_2[0][np.where(file_2[1] == max(file_2[1]))[0][0]]
    np.save(f'{path_save_player}CK_Win.npy',matran_Win)
        
    for buoc3 in range(2*number_tran-1):
        list_player = [playerRandom1]*2 + [test] + [random_player]*(amount_player()-3)
        kq, file_ = normal_main(list_player,1500, [0])
        file_.append(file_2[0][np.where(file_2[1] == max(file_2[1]))[0][0]])
        np.save(f'{path_save_player}CK_Matran.npy',file_)
        list_player= [playerScore1]*3 + [random_player]*(amount_player()-3)
        kq, file_2 = normal_main(list_player,3500, [0])
        matran_Win= file_2[0][np.where(file_2[1] == max(file_2[1]))[0][0]]
        np.save(f'{path_save_player}CK_Win.npy',matran_Win)
    return matran_Win


